In [ ]:
from joblib import Parallel, delayed
def enhance_alpha(im, mask, row_split=2, col_split=2, n_jobs=8):
	nrow, ncol = im.shape[:2]

	slices = []
	for i in range(row_split):
		for j in range(col_split):
			slices.append((slice(i*nrow//row_split, (i+1)*nrow//row_split),
							slice(j*nrow//col_split, (j+1)*nrow//col_split)))

	patches = Parallel(n_jobs=n_jobs)(delayed(other_repo_function)(im[ss].copy(), mask[ss].copy()) for ss in slices)

	for i, ss in enumerate(slices):
		mask[ss] = patches[i]

	return mask


In [1]:
import cv2
import numpy as np
from PIL import Image
from PIL import ImageFilter
from pymatting import cutout
from matting import alpha_matting, load_image, save_image, estimate_foreground_background, stack_images

from scipy import ndimage
from scipy import misc
import scipy.misc
import scipy
import image_slicer
from image_slicer import join
import matplotlib.pyplot as plt
import time
from copy import deepcopy
import asyncio


# Trying alpha-matting now 

num_tiles = 16
img = 'ex_3/original_big.jpg'
tiles = image_slicer.slice(img, num_tiles, save=True)
mask = 'ex_3/mask_big.png'
amask = cv2.cvtColor(cv2.imread(mask), cv2.COLOR_BGR2GRAY)
mtiles = image_slicer.slice("ex_3/conv_mask.png", num_tiles, save=False)
ftiles = deepcopy(tiles)



def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def para_alpha(i):
    start_time = time.time()

#     print(i)
    image = np.array(tiles[i].image)[:, :, [0, 1, 2]] / 255.0
    trimap = np.array(mtiles[i].image) / 255.0
#     alpha = alpha_matting(image, trimap, method="ifm", preconditioner="vcycle", print_info=False)
    foreground, background = estimate_foreground_background(image, trimap, print_info=False)

    cutout = stack_images(foreground, trimap)
    cutout = np.clip(cutout * 255, 0, 255).astype(np.uint8)
    ftiles[i].image = Image.fromarray(cutout)


    save_image("stitch/plant_cutout_big_" + str(tiles[i].basename.split('_', 2)[-1]) + ".png", cutout)
    print("Time taken for slice: ", i, ":",time.time() - start_time)
    #code

for i in range(num_tiles):
    para_alpha(i)


output = join(ftiles)

output.save('output.png')

In [ ]:
np.array(ftiles[0].image).shape[:2]

In [ ]:
ftiles[3].image

In [ ]:
into = cv2.imread('stitch/plant_cutout_big_01_01.png', cv2.IMWR)

In [ ]:
cv2.imwrite('checking.png', into)

In [ ]:
into.shape

In [ ]:
# plt.imshow(cv2.cvtColor(orig_2, cv2.COLOR_BGR2RGB))

plt.imshow(cv2.cvtColor(cv2.imread('stitch/plant_cutout_big_01_01.png', cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(np.array(mtiles[0].image) , cmap='gray')

In [ ]:
join(ftiles[0].image).save('checking.png')

In [ ]:
cutout = np.clip(cutout * 255, 0, 255).astype(np.uint8)

In [ ]:
plt.imshow(np.array(cutout[:, :, [0, 1, 2]] * 255, dtype=int))

In [ ]:
Image.fromarray(cutout)

In [ ]:
type(Image.fromarray(np.array(cutout[:, :, [0, 1, 2]] * 255.0, dtype=int)))

In [ ]:
type(ftiles[0].image)

In [ ]:
ftiles[0].image = ftiles[1].image

In [ ]:
ftiles = image_slicer.slice(img, num_tiles, save=True)


In [ ]:
ttiles = image_slicer.open_images_in('stitch')

In [ ]:
for i in range(len(num))
image = load_image("ex_2/original.jpg", "RGB")
trimap = load_image("ex_2/mask copy.png", "GRAY")

alpha = alpha_matting(image, trimap, method="cf", preconditioner="vcycle", print_info=False)

foreground, background = estimate_foreground_background(image, alpha, print_info=False)

cutout = stack_images(foreground, alpha)

save_image("plant_cutout.png", cutout)


In [ ]:
trimap = load_image("ex_3/mask_big.png", "GRAY", 0.5, "nearest")
trimap[trimap>=0.8] = 1
trimap[trimap<=0.2] = 0
trimap[(trimap > 0.2) & (trimap < 0.8)] = 0.5

np.unique(trimap)

from pymatting import *
import numpy as np
import scipy.sparse

scale = 1.0


image = load_image("ex_3/original_big.jpg", "RGB", scale, "box")
trimap = load_image("ex_3/mask_big.png", "GRAY", scale, "nearest")
trimap[trimap>=0.7] = 1
trimap[trimap<=0.3] = 0
trimap[(trimap > 0.2) & (trimap < 0.8)] = 0.5

# height and width of trimap
h, w = trimap.shape[:2]

# calculate laplacian matrix
L = cf_laplacian(image)

# decompose trimap
is_fg, is_bg, is_known, is_unknown = trimap_split(trimap)

# constraint weight
lambda_value = 100.0

# build constraint pixel selection matrix
c = lambda_value * is_known
C = scipy.sparse.diags(c)

# build constraint value vector
b = lambda_value * is_fg

# build linear system
print("Shape of L:", L.shape)
print("Shape of C: ", C.shape)
A = L + C

# build ichol preconditioner for faster convergence
A = A.tocsr()
A.sum_duplicates()
M = ichol(A)

# solve linear system with conjugate gradient descent
x = cg(A, b, M=M)

# clip and reshape result vector
alpha = np.clip(x, 0.0, 1.0).reshape(h, w)

save_image("lemur_alpha.png", alpha)

In [ ]:
test = cv2.imread('old_matting/examples/plant_image.jpg')

In [ ]:
test.shape

# Reading Images 

In [ ]:
threshold = 100 #Threshold for masking

# orig = cv2.imread('ex_1/Raquib_pic.jpg')
# amask = cv2.imread('ex_1/MASK.png')
orig = cv2.imread('ex_2/original.jpg')
# amask = cv2.imread('ex_2/mask copy.png')
amask = cv2.imread('ex_2/mask-our-site.png')

# orig = cv2.imread('ex_3/original_big.jpg')
# amask = cv2.imread('ex_3/mask_big.png')


if orig.shape == amask.shape:
    orig_2 = orig.copy()
else:
    orig_2 = cv2.resize(orig, (amask.shape[1], amask.shape[0]), interpolation = cv2.INTER_CUBIC) 

amask[amask<threshold] = 0
amask[amask>=threshold] = 255

In [ ]:
amask.shape

In [ ]:
np.unique(trimap, return_counts=True)

In [ ]:
threshold/255

In [ ]:
from pymatting import *
import numpy as np
import scipy.sparse

scale = 1.0

image = load_image("ex_2/original.jpg", "RGB", scale, "box")
trimap = load_image("ex_2/mask copy.png", "GRAY", scale, "nearest")
# trimap[trimap<threshold/255] = 0
# trimap[trimap>=threshold/255] = 255

# height and width of trimap
h, w = trimap.shape[:2]

# calculate laplacian matrix
L = cf_laplacian(image)

# decompose trimap
is_fg, is_bg, is_known, is_unknown = trimap_split(trimap)

# constraint weight
lambda_value = 100.0

# build constraint pixel selection matrix
c = lambda_value * is_known
C = scipy.sparse.diags(c)

# build constraint value vector
b = lambda_value * is_fg

# build linear system
A = L + C

# build ichol preconditioner for faster convergence
A = A.tocsr()
A.sum_duplicates()
M = ichol(A)

# solve linear system with conjugate gradient descent
x = cg(A, b, M=M)

# clip and reshape result vector
alpha = np.clip(x, 0.0, 1.0).reshape(h, w)

save_image("lemur_alpha.png", alpha)

In [ ]:
amask.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.axis("off")
# plt.imshow(cv2.cvtColor(mask2, cv2.COLOR_BGR2RGB))
# plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
# plt.imshow(cv2.cvtColor(orig_2, cv2.COLOR_BGR2RGB))
# plt.imshow(cv2.GaussianBlur(mask,(7,7),0))
plt.imshow(amask, cmap='gray')

mask = amask.copy()
# mask = cv2.GaussianBlur(mask,(7,7),0)


In [ ]:
kernel = np.ones((1,1), np.uint8) 
  
img_erosion = cv2.erode(amask, kernel, iterations=1) 
img_dilation = cv2.dilate(amask, kernel, iterations=1) 


In [ ]:
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(img_erosion)

In [ ]:
mask = img_erosion

In [ ]:
sharpened1 = pil_mask.filter(ImageFilter.SHARPEN)
sharpened2 = sharpened1.filter(ImageFilter.SHARPEN)
sharpened3 = sharpened2.filter(ImageFilter.SHARPEN)


kernel = np.ones((1, 1), np.uint8) 
  
# Using cv2.erode() method  
mask = cv2.erode(amask, kernel)


sharpened3.show()

In [ ]:
# actual = Image.fromarray(orig_2)
# actual.putalpha(0)

# actual.show()

In [ ]:
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

In [ ]:
orig_2 = np.uint8(orig_2)
mask = np.uint8(mask)

In [ ]:
# src1_mask=cv2.cvtColor(src1_mask,cv2.COLOR_GRAY2BGR)#change mask to a 3 channel image 
# mask_out=cv2.subtract(src1_mask,src1)
# mask_out=cv2.subtract(src1_mask,mask_out)


mask_out=cv2.subtract(amask,orig_2)
mask_out=cv2.subtract(amask,mask_out)

In [ ]:
const = orig_2 * amask

In [ ]:
plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(-cv2.cvtColor(const, cv2.COLOR_BGR2RGB))

In [ ]:
pi_const = Image.fromarray(const)

In [ ]:
sharpened1 = pi_const.filter(ImageFilter.SHARPEN)
sharpened2 = sharpened1.filter(ImageFilter.SHARPEN)
sharpened3 = sharpened2.filter(ImageFilter.SHARPEN)


kernel = np.ones((1, 1), np.uint8) 
  
# Using cv2.erode() method  
mask = cv2.erode(amask, kernel)


im2 = pi_const.filter(ImageFilter.MinFilter(3))
im2.show()

In [ ]:
bmask = np.array(amask, dtype=bool)

In [ ]:
bmask.shape

In [ ]:
np.unique(amask, return_counts=True)

In [ ]:
np.unique(bmask, return_counts=True)

In [ ]:
(orig_2*bmask).shape

In [ ]:
orig_2[amask].reshape(orig_2.shape)

In [ ]:
plt.figure(figsize=(10, 10))
plt.axis("off")
# plt.imshow(cv2.cvtColor(cv2.bitwise_or(orig_2, orig_2, mask=mask), cv2.COLOR_BGR2RGB))
plt.imshow(cv2.cvtColor((orig_2*bmask), cv2.COLOR_BGR2RGB))

In [ ]:
ano = Image.fromarray(cv2.cvtColor(cv2.bitwise_or(cv2.bitwise_or(orig_2, orig_2, mask=mask), cv2.bitwise_or(orig_2, orig_2, mask=mask), mask=mask) , cv2.COLOR_BGR2RGB))